# Выделение списка условий из вакансии

In [81]:
import sys
sys.path.append('./GOT/')
from got.asts import ast as ff
from got.asts import base as base

In [82]:
import pandas as pd
import re
import numpy as np

In [83]:
def get_picked(filename):
    f = open(filename, mode='r', encoding='UTF-8')
    picked = f.readlines()
    f.close()
    picked = list(map(lambda x: x.replace('\n', '').lower(), picked))
    return picked

In [84]:
def clear_text(text, lowerize=True):
    pat = re.compile(r'[^A-Za-z0-9 \-\n\r.,;!?А-Яа-я]+')
    cleared_text = re.sub(pat, ' ', text)

    if lowerize:
        cleared_text = cleared_text.lower()

    tokens = cleared_text.split()
    return tokens

In [85]:
def make_substrings(tokens, k=4):
    for i in range(max(len(tokens) - k + 1, 1)):
        yield ' '.join(tokens[i:i + k])


def get_relevance_matrix(texts, strings):
    matrix = np.empty((0, len(strings)), float)
    prepared_text_tokens = [clear_text(t) for t in texts]

    prepared_string_tokens = [clear_text(s) for s in strings]
    prepared_strings = [' '.join(t) for t in prepared_string_tokens]

    for text_tokens in prepared_text_tokens:
        astt = base.AST.get_ast(list(make_substrings(text_tokens)))
        row = np.array([astt.score(s) for s in prepared_strings])
        matrix = np.append(matrix, [row], axis=0)

    return matrix

In [86]:
def extract_text(data, column_name):
    texts = []
    for index, row in data.iterrows():
        text = row[column_name]
        if pd.notnull(text):
            if text.strip() == '':
                text = 'none'
            texts.append(text)
        else:
            texts.append('none')
    return texts

In [87]:
def save_relevance_matrix2(filename):
    np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
    with open(filename, 'w') as f:
        f.write(np.array2string(relevance_matrix, separator=', '))
    np.savetxt(filename, relevance_matrix)

In [88]:
data = pd.read_csv('data_forprint.csv', encoding='utf8', index_col=None)
data= data.drop('Unnamed: 0',axis=1)
data.head(5)

,domain_id,domain_name,profearea_id,profearea_name,ad_id,name,requirements,responsibilities,conditions
0,7,"Информационные технологии, интернет, телеком",107,Руководитель проектов,89187718,Руководитель проектов сопровождения,Умеете работать с рисками; \n Способны выстраи...,Развивать бизнес-приложения и решения внешних ...,Официальное оформление с первого рабочего дня;...
1,7,"Информационные технологии, интернет, телеком",10,Аналитик,90478414,Системный аналитик,Опыт работы аналитиком от 2-х лет; \nОпыт прое...,Возможность построить карьеру в ИТ банковской ...,Опыт работы с CRM-системами (Microsoft Dynamic...
2,7,"Информационные технологии, интернет, телеком",10,Аналитик,87797674,Ведущий аналитик,Управление работой группы аналитиков на проект...,Опыт проектирования систем различного класса в...,Никаких Дресс-кодов и стандартов \nКлассные ве...
3,7,"Информационные технологии, интернет, телеком",96,"Программист, разработчик",90037853,Программист Баз Данных (SQL),Уверенные знания SQL (MySQL) понимание плана в...,"Оптимизация SQL-кода (план запроса, управление...",Занятия в фитнес-клубе за счет компании; \nОпл...
4,7,"Информационные технологии, интернет, телеком",96,"Программист, разработчик",88043841,Разработчик в команду надёжности и производите...,знаете С++ и многопоточное программирование; \...,"придумывать, разрабатывать и улучшать инструме...",знаете ROS; \nпрограммировали и анализировали ...


In [89]:
texts = extract_text(data, 'conditions')

In [90]:
texts[0:4]

['Официальное оформление с первого рабочего дня; \nКомпенсация расходов на спорт и здоровье или курсы английского языка; \nУдобный график работы с гибким началом рабочего дня; \nКонкурентная оплату (оклад + бонус по результатам выполнения проектов); \n<strong>Возможность удаленной работы;</strong> \nСистема грейдов, систему наставничества, индивидуальный план развития; \nРабота среди профессионалов, наша команда одна из самых компетентных команд по внедрению ECM на российском рынке; \nРегулярные образовательные мероприятия внутри компании, участие и выступления на конференциях. ',
 'Опыт работы с CRM-системами (Microsoft Dynamics 365, Сreatio, Oracle Siebel CRM, Salesforce и т.д.); \nОпыт работы в системных интеграторах/вендорах. ',
 'Никаких Дресс-кодов и стандартов \nКлассные вечеринки (Отдых — это важная часть работы) \nВысокий уровень дохода (в том числе бонусы за результат) \nМедицинскую страховку (ДМС, включая стоматологию и госпитализацию, страхование выезжающих за рубеж) \nКорп

In [91]:
 picked = get_picked("cond_picked.txt")

In [92]:
picked

['стоматологией',
 'дмс',
 'обучение',
 'официальный',
 'компенсация',
 'спорт',
 'курсы',
 'английский',
 'гибкий',
 'заработная плата',
 'тк рф',
 'работы 5 2',
 'карьерного роста',
 'аккредитованной',
 'it компании',
 'гибкое начало',
 'удаленный формат',
 'корпоративные мероприятия',
 'дмс стоматологией',
 'удаленный формат работы',
 'гибкое начало рабочего дня',
 'бонус результатам',
 'система грейдов систему',
 'систему наставничества индивидуальный',
 'индивидуальный план развития',
 'профессионалов наша команда',
 'регулярные образовательные мероприятия',
 'microsoft dynamics 365',
 'услуги корпоративного такси',
 'график работы 5 2',
 '9 00 18',
 'дмс после испытательного срока',
 'страховку несчастных случаев',
 'льготное страхование вашим близким',
 'премии результатам работы',
 'тренинги вебинары базы знаний',
 'точек роста карьерном развитии',
 'стабильная зп оклад премия',
 'компании льготная ипотека отсрочка',
 'развития тинькофф апгрейд курсы тренинги',
 'акция приведи 

# Построение матрицы релевантности вакансия - фраза из "очищенного" списка условий работы

In [93]:
relevance_matrix = get_relevance_matrix(texts, picked)

In [94]:
np.savetxt('relevance_matrix_cond.txt', relevance_matrix)

In [95]:
mask = relevance_matrix >= 0.3
idx_x_conditions = pd.DataFrame(columns=['ad_id', 'conditions_refined', 'cond_idxs'])
for i in range(len(mask)):
    idx_x_conditions = idx_x_conditions._append({'ad_id': data.loc[i, 'ad_id'], 'conditions_refined': np.array(picked)[mask[i]],'cond_idxs': np.where(relevance_matrix[i] >= 0.3)}, ignore_index=True)


In [96]:
idx_x_conditions[0:4]

,ad_id,conditions_refined,cond_idxs
0,89187718,"[официальный, компенсация, английский, it комп...","([3, 4, 7, 14, 17, 20, 21, 22, 23, 24, 25, 26,..."
1,90478414,"[работы 5 2, система грейдов систему, microsof...","([11, 22, 27],)"
2,87797674,"[стоматологией, обучение, английский, it компа...","([0, 2, 7, 14, 17, 18, 21, 28, 43, 48],)"
3,90037853,"[обучение, it компании, гибкое начало, гибкое ...","([2, 14, 15, 20, 31],)"


In [97]:
final = pd.merge(data, idx_x_conditions, left_on='ad_id', right_on='ad_id', how='left')
final[['conditions', 'conditions_refined']]
final[['profearea_name', 'ad_id', 'conditions', 'conditions_refined', 'cond_idxs']].to_csv('data_with_cond_refined.csv')

In [98]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
save_relevance_matrix2('relevance_matrix_cond_2.txt')

# Выделение списка требований из вакансии

In [99]:
texts = extract_text(data, 'requirements')

In [100]:
texts[0:4]

['Умеете работать с рисками; \n Способны выстраивать эффективные переговоры и деловую переписку;  \n Понимаете отличия Agile от Waterfall и условия, когда нужно применять каждый из них.  ',
 'Опыт работы аналитиком от 2-х лет; \nОпыт проектирования реляционных БД; \nУверенное проектирование ER-диаграмм; \nОпыт проектирования API; \nОпыт проектирования пользовательских интерфейсов; \nОпыт проектирования бизнес-процессов в нотации BPMN; \nУмение самостоятельно искать информацию, предлагать варианты решений. ',
 'Управление работой группы аналитиков на проекте (2-5 человек) \nКонтроль качества работы аналитиков на проекте \nОпределение общих подходов к выполнению системного/бизнес-анализа в команде, определение базовых артефактов \nСбор, анализ и согласование требований с заказчиками \nРазработка и согласование проектной документации (описание бизнес-процессов, их реализации и необходимых интеграций, инструкций) \nПостановка задач разработчикам, приемка результатов \nПроведение приемо-сда

In [101]:
 picked = get_picked("req_picked.txt")

In [102]:
picked

['опыт',
 'знание',
 'понимание',
 'sql',
 '1с',
 'образование',
 'система',
 'разработка',
 'технический',
 'опыт разработки',
 'понимание принципов',
 'высшее образование',
 'техническое образование',
 'опыт работы',
 '3 лет',
 'работать команде',
 'опыт написания',
 'опыт администрирования',
 'базовые знания',
 'высшее техническое образование',
 'аналитический склад ума',
 'опыт работы системами',
 'работы базами данных',
 'понимание принципов работы',
 'знание принципов построения',
 'описания бизнес процессов',
 'уверенный пользователь пк',
 'чтения технической документации',
 'опыт коммерческой разработки',
 'грамотная устная письменная речь',
 'знание английского языка уровне',
 'опыт работы базами данных',
 'опыт работы системами мониторинга',
 'уровне чтения технической документации',
 'желание развиватся данном направлении',
 'опыт работы системным аналитиком',
 'стека протоколов tcp ip',
 'опыт работы docker',
 'настройке операционных систем офисных',
 'клиентов голосовые об

# Построение матрицы релевантности вакансия - фраза из "очищенного" списка  требований

In [103]:
relevance_matrix = get_relevance_matrix(texts, picked)

In [104]:
np.savetxt('relevance_matrix_req.txt', relevance_matrix)

In [105]:
mask = relevance_matrix >= 0.3
idx_x_conditions = pd.DataFrame(columns=['ad_id', 'requiremnets_refined', 'req_idxs'])
for i in range(len(mask)):
    idx_x_conditions = idx_x_conditions._append({'ad_id': data.loc[i, 'ad_id'], 'requirements_refined': np.array(picked)[mask[i]],'req_idxs': np.where(relevance_matrix[i] >= 0.3)}, ignore_index=True)


In [106]:
idx_x_conditions[0:4]

,ad_id,requiremnets_refined,req_idxs,requirements_refined
0,89187718,NaN,"([15],)",[работать команде]
1,90478414,NaN,"([0, 13, 25, 26, 35],)","[опыт, опыт работы, описания бизнес процессов,..."
2,87797674,NaN,"([7, 15, 25, 35],)","[разработка, работать команде, описания бизнес..."
3,90037853,NaN,"([1, 2, 3, 13, 23],)","[знание, понимание, sql, опыт работы, понимани..."


In [107]:
final = pd.merge(data, idx_x_conditions, left_on='ad_id', right_on='ad_id', how='left')
final[['requirements', 'requirements_refined']]
final[['profearea_name', 'ad_id', 'requirements', 'requirements_refined', 'req_idxs']].to_csv('data_with_req_refined.csv')

In [108]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
save_relevance_matrix2('relevance_matrix_req_2.txt')

# Выделение списка обязанностей из вакансии

In [109]:
texts = extract_text(data, 'responsibilities')

In [110]:
texts[0:4]

['Развивать бизнес-приложения и решения внешних заказчиков; \nУчаствовать в оценке запросов клиентов на сопровождении и согласовывать договоры; \n Разрабатывать план проекта и документацию;  \nВыполнять ключевые показатели эффективности и качества работы команды; \nВести внутреннюю и внешнюю отчетность по работе команды; \nПодбирать и развивать участников команды сопровождения; \n Формировать и развивать долгосрочные деловые отношения с клиентами.  ',
 'Возможность построить карьеру в ИТ банковской сферы; \nРабота в дружной команде; \nСоздание продукта на современном микросервисном стеке; \nУчастие в развитии low-code и даже no-code подходов; \nРеальный опыт проектирования решений для банковской отрасли; \nРешение сложных и интересных архитектурных вопросов; \nДрайв клиентских проектов; \nВозможность стать лидером команды; \nРабота с крупнейшими банками России. ',
 'Опыт проектирования систем различного класса в качестве самостоятельного аналитика, а также ведущего аналитика \nОпыт про

In [119]:
 picked = get_picked("resp_picked.txt")

In [120]:
picked

['разработка',
 'проект',
 'решение',
 'контроль',
 'тестирование',
 'участие',
 'взаимодействие',
 'документация',
 'контроль',
 'тестирование',
 'бизнес процессов',
 'постановка задач',
 'установка настройка',
 'участие разработке',
 'платформе 1с',
 'программного обеспечения',
 'проектной документации',
 'информационных систем',
 'технических заданий',
 'информационной безопасности',
 'сбор анализ',
 'принимать входящие звонки',
 'постановка задач разработчикам',
 'анализ бизнес процессов',
 'настройка windows офисных',
 'консультирование пользователей',
 'разработка нового функционала',
 'быструю систему адаптации',
 'обеспечение бесперебойной работы',
 'подключение настройка периферийного оборудования',
 'сборка размещение комплектов оборудования',
 'чаты 40 звонки скрипт',
 'диагностика проведение ремонта профилактики',
 'принимать входящие звонки обращения',
 'вносить информацию специализированную программу',
 'сборка размещение комплектов оборудования',
 'отчетности в мобильном

# Построение матрицы релевантности вакансия - фраза из "очищенного" списка обязанностей

In [121]:
relevance_matrix = get_relevance_matrix(texts, picked)

In [122]:
np.savetxt('relevance_matrix_resp.txt', relevance_matrix)

In [123]:
mask = relevance_matrix >= 0.3
idx_x_conditions = pd.DataFrame(columns=['ad_id', 'responsibilities_refined', 'resp_idxs'])
for i in range(len(mask)):
    idx_x_conditions = idx_x_conditions._append({'ad_id': data.loc[i, 'ad_id'], 'responsibilities_refined': np.array(picked)[mask[i]],'resp_idxs': np.where(relevance_matrix[i] >= 0.3)}, ignore_index=True)


In [124]:
idx_x_conditions[0:4]

,ad_id,responsibilities_refined,resp_idxs
0,89187718,"[разработка, проект, решение, документация, уч...","([0, 1, 2, 7, 13, 16, 36],)"
1,90478414,"[проект, решение, тестирование, участие, тести...","([1, 2, 4, 5, 9],)"
2,87797674,"[разработка, проект, тестирование, взаимодейст...","([0, 1, 4, 6, 9, 17, 39],)"
3,90037853,"[разработка, взаимодействие, бизнес процессов,...","([0, 6, 10, 17, 23, 26, 39],)"


In [125]:
final = pd.merge(data, idx_x_conditions, left_on='ad_id', right_on='ad_id', how='left')
final[['responsibilities', 'responsibilities_refined']]
final[['profearea_name', 'ad_id', 'responsibilities', 'responsibilities_refined', 'resp_idxs']].to_csv('data_with_resp_refined.csv')

In [126]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
save_relevance_matrix2('relevance_matrix_resp_2.txt')